In [ ]:
from flask import Flask, render_template, redirect, url_for, request, flash, session
import pandas as pd
from flask_sqlalchemy import SQLAlchemy
from flask_login import UserMixin, login_user, LoginManager, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from flask_bcrypt import check_password_hash
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Length, ValidationError
from wtforms.fields.html5 import DateField
from datetime import datetime

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///user.db'
app.config['SECRET_KEY'] = "thisisasecretkey"
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class User(db.Model, UserMixin):
    __tablename__ = 'user'
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(20), unique=True, nullable = False)
    password = db.Column(db.String(80), nullable = False)
    email = db.Column(db.String(50), nullable = False)
    todo = db.relationship('Todo', backref='writer', lazy='dynamic')

class Todo(db.Model):
    __tablename__ = 'todo'
    id = db.Column(db.Integer, primary_key=True, nullable=False)
    content = db.Column(db.String(200), nullable=False)
    date_created = db.Column(db.DateTime, default=datetime.utcnow)
    #date_created = db.Column(db.String(200), nullable=False)
    writer_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    
class RegisterForm(FlaskForm):
    username = StringField(validators=[InputRequired(), Length(min=4, max=20)], render_kw={"placeholder":"Username"})
    password = PasswordField(validators=[InputRequired(), Length(min=4, max=20)], render_kw={"placeholder": "Password"})
    email = StringField(validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "Email address"})
    submit = SubmitField("Register")
    
    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")

class LoginForm(FlaskForm):
    username = StringField(validators=[InputRequired(), Length(min=4, max=20)], render_kw={"placeholder":"Username"})
    password = PasswordField(validators=[InputRequired(), Length(min=4, max=20)], render_kw={"placeholder": "Password"})
    submit = SubmitField("Login")

class NewNoteForm(FlaskForm):
    content = StringField(validators=[InputRequired(), Length(max=150)], render_kw={"placeholder": "Content"})
    #date_created = StringField(validators=[InputRequired(), Length(max=150)], render_kw={"placeholder": "Date"})
    date_created = DateField('DatePicker', format='%Y-%m-%d')
    submit = SubmitField("Add Note")
    
@app.route("/")
def index():
    return render_template('index.html')

@app.route("/health")
def health():
    return render_template('health.html')

@app.route("/about")
def about():
    return render_template('about.html')
    
@app.route("/register", methods=["GET", "POST"])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, password=hashed_password, email=form.email.data)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', form=form)

@app.route("/login", methods=["GET", "POST"])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for('dashboard'))
        flash ("User does not exist, or invalid username or password.")
    return render_template('login.html', form=form)

@app.route("/dashboard", methods=["GET", "POST"])
@login_required
def dashboard():
    form = NewNoteForm()
    if form.validate_on_submit():
        print("form submitted")
        new_note = Todo(content=form.content.data, date_created=form.date_created.data, writer=current_user)
        db.session.add(new_note)
        db.session.commit()
        return redirect(url_for('mynote'))
    print("successfully submit")
    return render_template('dashboard.html', form=form)
    
@app.route('/mynote', methods=['GET','POST'])
@login_required
def mynote():
    todos = Todo.query.filter_by(writer = current_user).all()
    return render_template('mynote.html', todos=todos, title='My Notes')

@app.route('/delete/<int:id>', methods=['GET', 'POST'])
def delete(id):
    task_to_delete = Todo.query.get_or_404(id) 
    db.session.delete(task_to_delete)
    db.session.commit()
    return redirect(url_for('mynote'))

@app.route('/update/<int:id>', methods=['GET', 'POST'])  
def update(id):
    task = Todo.query.get_or_404(id)

    if request.method == 'POST':
        task.content = request.form['content']

        try:
            db.session.commit()
            return redirect(url_for('mynote'))
        except:
            return 'There was an issue updating your task'

    else:
        return render_template('update.html', task=task)

@app.route("/logout", methods=["GET", "POST"])
@login_required
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

if __name__ == '__main__':
    app.run('localhost', 9005)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\Users\User\anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9005/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Nov/2021 01:49:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 01:49:39] "GET /register HTTP/1.1" 200 -
